# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [50]:
# import json
import re
import pandas
# import numpy
import sklearn.tree
import sklearn.neighbors
import sklearn.linear_model
from sklearn.model_selection import cross_validate
from scipy import stats

# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10
0,6,184 m/(s^2),0.2578,TENNIS,fabrice,636,536 m^3,880.0,0.5333,0.0873,404,0.1197
1,4,368 m/(s^2),0.1664,golf,Tony,606,316 m^3,286.0,0.2354,0.4192,761,0.8783
2,1,647 m/(s^2),1.2483,motor sports,Reilly,237,33 m^3,421.0,0.616,0.0399,371,0.5624
3,0,160 m/(s^2),0.4863,ice hockey,Vani,253,561 m^3,480.0,0.0661,0.7897,235,0.6138
4,0,431 m/(s^2),0.5611,ice hockey,Vani,214,532 m^3,579.0,0.032,0.7095,143,0.6463
...,...,...,...,...,...,...,...,...,...,...,...,...
1548,2,1204 m/(s^2),0.4531,football,Reilly,-11,206 m^3,844.0,0.875,0.8171,1091,0.0238
1549,2,1126 m/(s^2),0.2987,football,Reilly,-79,213 m^3,794.0,0.8289,0.7422,1269,0.1006
1550,3,631 m/(s^2),0.2809,basketball,Andrew,372,1015 m^3,80.0,0.4277,0.2612,865,0.2983
1551,5,146 m/(s^2),0.703,running,Lise,1178,802 m^3,582.0,0.4904,0.994,418,0.5464


Don't forget to checkout the column information.

In [37]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1553 non-null   int64  
 1   col_00  1541 non-null   object 
 2   col_01  1544 non-null   object 
 3   col_02  1540 non-null   object 
 4   col_03  1538 non-null   object 
 5   col_04  1540 non-null   object 
 6   col_05  1543 non-null   object 
 7   col_06  1544 non-null   float64
 8   col_07  1545 non-null   object 
 9   col_08  1537 non-null   object 
 10  col_09  1540 non-null   object 
 11  col_10  1533 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 145.7+ KB


And any numeric information.

In [38]:
unique_data.describe()

,label,col_06
count,1553.000000,1544.000000
mean,2.977463,495.709197
std,1.994227,294.362437
min,0.000000,-294.000000
25%,1.000000,258.000000
50%,3.000000,497.500000
75%,5.000000,740.250000
max,6.000000,1132.000000


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [51]:
def clean_data(frame):
    size = len(frame)
    pandas.set_option('mode.chained_assignment', None)

    # clean numerical values by removing non-numerical parts of strings and cleaning non-numeric
    # values by making them all lowercase
    for column in frame.columns:
        null_indices = frame[column].isna()
        for index in range(size):
            question_mark = re.search('[?	]', str(frame[column][index]).strip())
            char_match = re.search('[a-z]+', str(frame[column][index]))
            unit_squared_match = re.search(r'[-]?\d+[ ]?(m\/\(s\^2\))',
                                           str(frame[column][index]).strip())
            unit_cubed_match = re.search(r'[-]?\d+[ ]?(m\^3)', str(frame[column][index]).strip())
            digit_match = re.search(r'\d+', str(frame[column][index]))
            
            if (digit_match):
                frame[column][index] = re.sub(r'[^0-9\.]', "", str(frame[column][index]))
                if (char_match and (unit_squared_match or unit_cubed_match)):
                    if (len(str(frame[column][index])) == 1):
                        frame[column][index] = "0"
                    else:
                        str_ver = str(frame[column][index])
                        frame[column][index] = (str_ver[:len(str(frame[column][index])) - 1])
            elif (question_mark or null_indices[index]):
                frame[column][index] = "0"
            else:
                frame[column][index] = (frame[column][index]).lower()

    # one-hot for non-numerical columns
    for column in frame.columns:
        char_match = re.search('[a-z]+', str(frame[column][index]))
        
        if (char_match):
            one_hot = pandas.get_dummies(frame[column], prefix = column, dtype=float)
            frame = pandas.concat([frame, one_hot], axis = 1)
            frame.pop(column)

    # dealing with column names that have a comma in them (e.g michael, lise)
    for column in frame.columns:
        comma_match = re.search(',', str(column))
        if (comma_match):
            col = column.split("_")
            col_name = "".join([col[0], "_", col[1]])
            just_values = column.replace("".join([col_name, "_"]), '')
            words = just_values.split(",")
            for word in words:
                for index in frame.index:
                    if (frame[column][index] == 1.0):
                        frame["".join([col_name, "_", word.strip()])][index] = 1.0
            frame.pop(column)

    # typing columns
    for column in frame.columns:
        null_indices = frame[column].isna()
        counts = frame.isnull().sum().sum()
        for index in frame.index:
            period_match = re.search(r'\d+[\.]\d+', str(frame[column][index]))
            digit_match = re.search(r'\d+', str(frame[column][index]))
            if (period_match and digit_match):
                frame[column] = frame[column].astype(float)
                break
            elif (digit_match):
                if (counts == 0):
                    frame[column] = frame[column].astype('int', errors = "ignore")
                else:
                    frame[column] = frame[column].astype(float)
    return frame

unique_data = clean_data(unique_data)
unique_data

/tmp/ipykernel_2373/4252672924.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  frame[column][index] = re.sub("[^0-9\.]", "",  str(frame[column][index]))
/tmp/ipykernel_2373/4252672924.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '880.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  frame[column][index] = re.sub("[^0-9\.]", "",  str(frame[column][index]))


,label,col_00,col_01,col_04,col_05,col_06,col_07,col_08,col_09,col_10,...,col_03_chris,col_03_eriq,col_03_fabrice,col_03_inan,col_03_lise,col_03_michael,col_03_reilly,col_03_sam,col_03_tony,col_03_vani
0,6,184,0.2578,636,536,880.0,0.5333,0.0873,404,0.1197,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,368,0.1664,606,316,286.0,0.2354,0.4192,761,0.8783,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,647,1.2483,237,33,421.0,0.6160,0.0399,371,0.5624,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,160,0.4863,253,561,480.0,0.0661,0.7897,235,0.6138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,431,0.5611,214,532,579.0,0.0320,0.7095,143,0.6463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,2,1204,0.4531,11,206,844.0,0.8750,0.8171,1091,0.0238,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1549,2,1126,0.2987,79,213,794.0,0.8289,0.7422,1269,0.1006,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1550,3,631,0.2809,372,1015,80.0,0.4277,0.2612,865,0.2983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1551,5,146,0.7030,1178,802,582.0,0.4904,0.9940,418,0.5464,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Now we should also be able to view all the numeric columns.

In [40]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   label                 1553 non-null   int64  
 1   col_00                1553 non-null   int64  
 2   col_01                1553 non-null   float64
 3   col_04                1553 non-null   int64  
 4   col_05                1553 non-null   int64  
 5   col_06                1553 non-null   float64
 6   col_07                1553 non-null   float64
 7   col_08                1553 non-null   float64
 8   col_09                1553 non-null   int64  
 9   col_10                1553 non-null   float64
 10  col_02_0              1553 non-null   float64
 11  col_02_badminton      1553 non-null   float64
 12  col_02_baseball       1553 non-null   float64
 13  col_02_basketball     1553 non-null   float64
 14  col_02_boxing         1553 non-null   float64
 15  col_02_football      

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [41]:
def create_classifiers():
    classifiers = [sklearn.linear_model.LogisticRegression(),
                   sklearn.tree.DecisionTreeClassifier(max_depth = 100, random_state = 0),
                   sklearn.neighbors.KNeighborsClassifier(n_neighbors = 3)]
    return classifiers

my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(),
 DecisionTreeClassifier(max_depth=100, random_state=0),
 KNeighborsClassifier(n_neighbors=3)]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [42]:
def cross_fold_validation(classifier, frame, folds):
    # prepare data
    df1 = frame.copy()

    X = df1[frame.columns].values
    y = df1.label.values
    scores = cross_validate(classifier, X, y, cv = folds)

    return list(scores['test_score'])

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

/home/fileung/cse40_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fileung/cse40_venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

Classifier: LogisticRegression, Accuracy: [0.9614147909967846, 0.9807073954983923, 0.9581993569131833, 0.9774193548387097, 0.9806451612903225].
Classifier: DecisionTreeClassifier, Accuracy: [1.0, 1.0, 1.0, 1.0, 1.0].
Classifier: KNeighborsClassifier, Accuracy: [0.9903536977491961, 0.9935691318327974, 0.9742765273311897, 0.9903225806451613, 0.9903225806451613].


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [49]:
def significance_test(a_values, b_values, p_value):
    if ((stats.ttest_ind(a_values, b_values)[1]) < p_value):
        return True
    else:
        return False
    
for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

0.00041933366954616854
LogisticRegression vs DecisionTreeClassifier: True
0.027626775622747155
LogisticRegression vs KNeighborsClassifier: True
0.007346297158511609
DecisionTreeClassifier vs KNeighborsClassifier: True


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.